In [1]:
import matplotlib.pyplot as plt
from glob import glob

import cv2
import os
import numpy as np
import time
import base64
import codecs, json
import pickle

from PIL import Image


# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
#from keras_retinanet.utils.gpu import setup_gpu
from keras.models import Model



# object detection
os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

model_path = os.path.join('./keras-retinanet', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')


Using TensorFlow backend.


In [2]:
dataset_path = "./original_test/"
output_path = "./detected_data/detected_from_test/"

In [3]:

retina_model = models.load_model(model_path, backbone_name='resnet50')
os.chdir(dataset_path)
dataset_list = os.listdir(os.getcwd())
os.chdir('../')


from object_detection import object_detection
object_detection(retina_model, dataset_list, dataset_path, output_path)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
handling person000355.png
image shape: (427, 640, 3)
resized image size: (800, 1199, 3) scale: 1.873536299765808

/home/plass-heesu/anaconda3/envs/tf1.15/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
(1, 300, 4) (1, 300) (1, 300)
processing time:  3.2125751972198486
handling

In [4]:
from ImageRetrievalClass import ImageRetrievalClass
from tqdm import tqdm

images_path = glob(output_path+"*.jpg")

for path in tqdm(images_path):
    image_pil = Image.open(path)
    image_resized = image_pil.resize((512,512))
    image_resized.save(path)
    

100%|██████████| 18/18 [00:00<00:00, 114.80it/s]


In [5]:
# try: "simpleAE", "convAE", "vgg19" , "IncepResNet", "ResNet50v2"
#     modelName = "IncepResNet"  # try: "simpleAE", "convAE", "vgg19" , "IncepResNet", "ResNet50v2"
#     trainModel = True
#     parallel = False  # use multicore processing

retrieval = ImageRetrievalClass("ResNet50v2", True, False)
retrieval.buildModel()
retrieval.transform()
retrieval.train()
E_train, E_test = retrieval.predict()



# image_retrieval(modelName="ResNet50v2",trainModel=True, parallel=False)

Reading train images from '/home/plass-heesu/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/detected_data/detected_from_train'...
Reading test images from '/home/plass-heesu/KimHeeSu/2021-1-CECD3-four-elSe-1/integrated_main/detected_data/detected_from_test'...


IndexError: list index out of range